In [1]:
import os
import json
import glob
import h5py
import numpy as np
from tqdm.notebook import tqdm

import sys
sys.path.append('../../py/')

from cosmoSim import cosmoSim

In [2]:
outdir = '../../data_prods/'
data_dir = '/home/ryan/Data'
subdir_list = [os.path.join(data_dir, o) for o in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir,o))]

print(subdir_list)

['/home/ryan/Data/run_2cDM_L3N256_DM_powerm2m2_sigma1_dir_2', '/home/ryan/Data/run_2cDM_L3N256_DM_power00_sigma1_dir_7_Vkick119.90', '/home/ryan/Data/run_2cDM_L3N256_DM_powerm2m2_sigma0.1', '/home/ryan/Data/run_2cDM_L3N256_DM_powerm2m2_sigma5.0_dir_9', '/home/ryan/Data/run_2cDM_L3N512_HY_power00_sigma1', '/home/ryan/Data/run_2cDM_L3N256_HY_power00_sigma0.1', '/home/ryan/Data/run_2cDM_L3N256_HY_power00_sigma1_dir_9', '/home/ryan/Data/run_SIDM_L3N256_DM_power0_sigma1', '/home/ryan/Data/run_2cDM_L3N256_HY_powerm2m2_sigma1_dir_5', '/home/ryan/Data/run_2cDM_L3N256_DM_powerm2m2_sigma1_dir_1', '/home/ryan/Data/lost+found', '/home/ryan/Data/run_2cDM_L3N256_HY_power00_sigma1_dir_5', '/home/ryan/Data/run_2cDM_L3N256_DM_power00_sigma1_dir_3', '/home/ryan/Data/run_2cDM_L3N256_HY_powerm2m2_sigma10', '/home/ryan/Data/run_2cDM_L3N256_HY_powerm2m2_sigma1_dir_8', '/home/ryan/Data/run_CDM_L3N256_DM_dir_7', '/home/ryan/Data/run_2cDM_L3N256_DM_power00_sigma1_dir_8', '/home/ryan/Data/run_CDM_L5N256_HY', '/

In [3]:
def get_masses(sbh_pos, rbins, coord, mass_table):

    r = (coord[:,0] - sbh_pos[0])**2 + (coord[:,1] - sbh_pos[1])**2 + (coord[:,2] - sbh_pos[2])**2

    m_in = []
    for R in rbins:
        m_in.append(mass_table[r <= R].sum())
    return np.array(m_in)
    

In [4]:
rewrite = False
max_no_subhalos = 10

nBins = 1000
N_partTypes = 6
massive_partTypes = [0, 1, 4, 5]

for subdir in tqdm(subdir_list):
    run_name = subdir.split('/')[-1]

    #reject folders that aren't runs
    if 'run' not in run_name:
        continue
    
    fpath = os.path.join(outdir, run_name)
    try:
        os.mkdir(fpath)
    except:
        print(f'{fpath} already exists!')

    run = cosmoSim(run_name)
    force_scale = 2.8 * run.softening_length

    snaps = sorted(glob.glob(subdir + '/snap*'))
    fofs = sorted(glob.glob(subdir + '/fof*'))

    print(f'Calculating mass densities for run {run_name}...')

    for i, (snap, fof) in enumerate(zip(snaps, fofs)):
        
        fout = os.path.join(fpath, f'subhalo_densities_{i}')
        try:
            os.mkdir(fout)
        except:
            print(f'{fout} already exists!')
        
        if not rewrite:
                if (len( glob.glob( os.path.join(fout, 'subhalo*.txt') ) ) >= max_no_subhalos):
                    print(f'Mass density files exist for {run_name}.')
                    print('Skipping...')
                    continue
        
        # get density for each subhalo
        with h5py.File(snap, "r") as f, h5py.File(fof, "r") as s:
            try:
                half_mass_radii = s["Subhalo/SubhaloHalfmassRad"][()]
            except:
                print(f'No subhalos formed for snapshot {i}...')
                continue
            
            subhalo = s.get('Subhalo')
            group = s.get('Group')

            no_subhalos = len(half_mass_radii)

            if no_subhalos > max_no_subhalos: no_subhalos = max_no_subhalos

            print(f'Calculating {no_subhalos} subhalo density profiles for snapshot {i}...')
            for subhalo_idx in range(no_subhalos):
                cm = subhalo['SubhaloCM'][subhalo_idx]
                max_rad = 3 * subhalo['SubhaloHalfmassRad'][subhalo_idx]
                nums = subhalo["SubhaloLenType"][subhalo_idx]
                group_idx = subhalo["SubhaloGrNr"][subhalo_idx]

                particles_before = np.zeros(N_partTypes).astype(int)
                if group_idx != 0:
                    # find all particles in groups before this group
                    particles_before += np.sum([ group['GroupLenType'][i] for i in range(group_idx) ], axis=0).astype(int)

                # which subhalo is this one within its group?
                first_in_group_idx = group["GroupFirstSub"][group_idx]
                # if -1, this is it
                if first_in_group_idx != -1:
                    # get particles for the previous subhalos
                    particles_before += np.sum([ subhalo['SubhaloLenType'][i] for i in range(first_in_group_idx, subhalo_idx) ], axis=0).astype(int)
                    
                particles_this = subhalo['SubhaloLenType'][subhalo_idx]

                rbins = np.geomspace(max_rad / 30, max_rad, num=nBins)

                total_mass = np.zeros(nBins)
                for partType in massive_partTypes:
                    
                    try:
                        coords = f.get(f"PartType{partType}/Coordinates")[()][particles_before[partType]:(particles_before[partType] + particles_this[partType])]
                        if partType != 1:
                            masses = f.get(f"PartType{partType}/Masses")[()][particles_before[partType]:(particles_before[partType] + particles_this[partType])]
                        else:
                            masses = f["Header"].attrs["MassTable"][1] * np.ones( particles_this[partType] ) 

                        total_mass += get_masses(cm, rbins, coords, masses)
                    except TypeError as e:
                        print( f'Particle Type {partType} does not exist...')
                        continue

                
                densities = total_mass / ( (4/3) * np.pi * rbins**3 )
                fname = os.path.join(fout, f'subhalo_{subhalo_idx}.txt')
                np.savetxt(fname, (rbins, densities))
                print(f'Saved subhalo {subhalo_idx} density profile at {fname}')

  0%|          | 0/154 [00:00<?, ?it/s]

../../data_prods/run_2cDM_L3N256_DM_powerm2m2_sigma1_dir_2 already exists!
Calculating mass densities for run run_2cDM_L3N256_DM_powerm2m2_sigma1_dir_2...
../../data_prods/run_2cDM_L3N256_DM_powerm2m2_sigma1_dir_2/subhalo_densities_0 already exists!
No subhalos formed for snapshot 0...
../../data_prods/run_2cDM_L3N256_DM_powerm2m2_sigma1_dir_2/subhalo_densities_1 already exists!
Mass density files exist for run_2cDM_L3N256_DM_powerm2m2_sigma1_dir_2.
Skipping...
../../data_prods/run_2cDM_L3N256_DM_powerm2m2_sigma1_dir_2/subhalo_densities_2 already exists!
Mass density files exist for run_2cDM_L3N256_DM_powerm2m2_sigma1_dir_2.
Skipping...
../../data_prods/run_2cDM_L3N256_DM_powerm2m2_sigma1_dir_2/subhalo_densities_3 already exists!
Mass density files exist for run_2cDM_L3N256_DM_powerm2m2_sigma1_dir_2.
Skipping...
../../data_prods/run_2cDM_L3N256_DM_powerm2m2_sigma1_dir_2/subhalo_densities_4 already exists!
Mass density files exist for run_2cDM_L3N256_DM_powerm2m2_sigma1_dir_2.
Skipping.

../../py/cosmoSim.py:54: UserWarning: Vkick not explicitly set! Assuming 100 km/s...
  warnings.warn('Vkick not explicitly set! Assuming 100 km/s...')


No subhalos formed for snapshot 0...
../../data_prods/run_2cDM_L3N512_HY_power00_sigma1/subhalo_densities_1 already exists!
Mass density files exist for run_2cDM_L3N512_HY_power00_sigma1.
Skipping...
../../data_prods/run_2cDM_L3N512_HY_power00_sigma1/subhalo_densities_2 already exists!
Mass density files exist for run_2cDM_L3N512_HY_power00_sigma1.
Skipping...
../../data_prods/run_2cDM_L3N512_HY_power00_sigma1/subhalo_densities_3 already exists!
Mass density files exist for run_2cDM_L3N512_HY_power00_sigma1.
Skipping...
../../data_prods/run_2cDM_L3N512_HY_power00_sigma1/subhalo_densities_4 already exists!
Mass density files exist for run_2cDM_L3N512_HY_power00_sigma1.
Skipping...
../../data_prods/run_2cDM_L3N512_HY_power00_sigma1/subhalo_densities_5 already exists!
Mass density files exist for run_2cDM_L3N512_HY_power00_sigma1.
Skipping...
../../data_prods/run_2cDM_L3N256_HY_power00_sigma0.1 already exists!
Calculating mass densities for run run_2cDM_L3N256_HY_power00_sigma0.1...
../..